# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [37]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [38]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [39]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# To get total number of rows 
print(len(full_data_rows_list))


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


171


In [40]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

156


# Part II. Complete the Apache Cassandra coding portion of this project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [41]:
# This connection is for a Cassandra instance in local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [42]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS playlist
    WITH REPLICATION =
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
""")

#### Set Keyspace

In [43]:
session.set_keyspace('playlist')


#### Create tables and queries to check the tables are well designed

##### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [44]:
drop_query = "DROP TABLE IF EXISTS songs_history"
session.execute(drop_query)
create_query = "CREATE TABLE IF NOT EXISTS songs_history "
create_query = create_query + """(sessionId int, itemInSession int, artist text, title text, length float, 
                                 PRIMARY KEY (sessionId, itemInSession))"""
session.execute(create_query)



# Provided part of the code to set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_history (sessionId, itemInSession, artist, title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

        
select_query = "SELECT artist, title, length FROM songs_history WHERE sessionId = 338 AND itemInSession = 4"
rows = session.execute(select_query)
for row in rows:
    print(row.artist, row.title, row.length)

##### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [45]:
drop_query = "DROP TABLE IF EXISTS song_played_by_user"
session.execute(drop_query)
create_query = "CREATE TABLE IF NOT EXISTS song_played_by_user "
create_query = create_query + """(userId int, sessionId int, itemInSession int, artist text, title text, firstName text, lastName text, 
                                 PRIMARY KEY((userId, sessionId), itemInSession))"""
session.execute(create_query)



file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_played_by_user (userId, sessionId, itemInSession, artist, title, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
  


select_query = "SELECT artist, title, firstName, lastName FROM song_played_by_user WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession DESC"
rows = session.execute(select_query)
for row in rows:
    print(row.artist, row.title, row.firstname, row.lastname)

                    

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz


##### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [46]:
drop_query = "DROP TABLE IF EXISTS users_by_song"
session.execute(drop_query)
create_query = "CREATE TABLE IF NOT EXISTS users_by_song "
create_query = create_query + """(title text, userId int, firstName text, lastName text, PRIMARY KEY (title, userId))"""
session.execute(create_query)



file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song (title, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        
        
        
select_query = "SELECT firstName, lastName FROM users_by_song WHERE title='All Hands Against His Own'"
rows = session.execute(select_query)
for row in rows:
    print(row.firstname, row.lastname)

                    

### Drop the tables before closing out the sessions

In [47]:
drop_query = "DROP TABLE IF EXISTS songs_history"
session.execute(drop_query)
drop_query = "DROP TABLE IF EXISTS song_played_by_user"
session.execute(drop_query)
drop_query = "DROP TABLE IF EXISTS users_by_song"
session.execute(drop_query)

### Close the session and cluster connection¶

In [48]:
session.shutdown()
cluster.shutdown()